# 2-models.ipynb
## Workflow: Slot-level clustering → Demand models → Price-response grids → Save artifacts


# A. Setup & Data Loading
# 1. Parameters & Libraries


In [6]:
import pandas as pd
import numpy as np
import os
import joblib

# Clustering
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

# Modeling
from sklearn.model_selection import train_test_split, TimeSeriesSplit, GridSearchCV
from sklearn.linear_model import LogisticRegression, PoissonRegressor
from sklearn.ensemble import RandomForestRegressor # Added as another strong baseline/alternative
import lightgbm as lgb
import catboost as cb
from sklearn.metrics import roc_auc_score, mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Calibration & Explainability
# from sklearn.calibration import calibration_curve # For classification
import shap

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Optuna for hyperparameter tuning
import optuna

# Disable Optuna's verbose logging during trials, but enable it for study summary
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Get the directory of the current script (code/)
SCRIPT_DIR = os.path.dirname(os.path.realpath("C:\Users\marti\OneDrive\Documentos\Martim\Académico\Master LBS\Clases MAM2025\LondonLAB\Clays_LondonLAB_Project"))

# Get the project root directory (CLAYS_LONDONLAB_Project/)
PROJECT_ROOT = os.path.abspath(os.path.join(SCRIPT_DIR, os.pardir))


# Define Parameters
SLOT_PARQUET = os.path.join(PROJECT_ROOT, "data", "processed", "slot_level.parquet")
CLUSTER_CSV = os.path.join(PROJECT_ROOT, "data", "processed", "time_slot_clusters.csv")
MODELS_DIR = os.path.join(PROJECT_ROOT, "models") # This matches your structure for a top-level 'models' folder
GRID_PATH = os.path.join(PROJECT_ROOT, "data", "processed", "price_response_grid.csv")

# Create directories if they don't exist
os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs("../data/processed/", exist_ok=True) # Ensure this exists too


# Print paths for verification (paths are now absolute)
print("Libraries imported and parameters set.")
print(f"Script directory: {SCRIPT_DIR}")
print(f"Project root: {PROJECT_ROOT}")
print(f"Models will be saved in: {MODELS_DIR}")
print(f"Cluster CSV will be saved to: {CLUSTER_CSV}")
print(f"Price Grid CSV will be saved to: {GRID_PATH}")
print(f"Loading slot data from: {SLOT_PARQUET}")


SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (800282122.py, line 37)

# 2. Load data

In [5]:
try:
    df_slot = pd.read_parquet(SLOT_PARQUET)
except FileNotFoundError:
    print(f"ERROR: Still cannot find the file at the constructed path: {SLOT_PARQUET}")
    print("Please double-check that the file exists and the path logic is correct for your environment.")
    raise # Re-raise the exception if it still occurs

print(f"Loaded data from {SLOT_PARQUET}. Shape: {df_slot.shape}")
df_slot.head()



ERROR: Still cannot find the file at the constructed path: C:\Users\marti\OneDrive\Documentos\Martim\Académico\Master LBS\Clases MAM2025\LondonLAB\data\processed\slot_level.parquet
Please double-check that the file exists and the path logic is correct for your environment.


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\marti\\OneDrive\\Documentos\\Martim\\Académico\\Master LBS\\Clases MAM2025\\LondonLAB\\data\\processed\\slot_level.parquet'

In [ ]:
# In[2] or equivalent in 2.py, after loading df_slot
df_slot = pd.read_parquet(SLOT_PARQUET)
print(f"Loaded data from {SLOT_PARQUET}. Shape: {df_slot.shape}")
print("Unique Venues in df_slot:", df_slot['Venue Name'].unique()) # <--- ADD THIS
if 'Birmingham' not in df_slot['Venue Name'].unique():
    print("WARNING: Birmingham is NOT present in the loaded slot_level.parquet.")
    print("This likely means it was filtered out in 1_ingest.ipynb, probably by the 'Search At >= 2024-08-01' filter.")
    print("Please check your raw data and the filters in 1_ingest.ipynb.")


In [ ]:
# Assert that it has all needed columns
required_cols = ['Venue Name', 'search_date_for', 'search_hour_for', 
                 'n_searches', 'n_bookings', 'booking_rate', 
                 'lead_time', 'day_of_week', 'is_weekend', 
                 'avg_price', 'pct_avail', 'capacity', 
                 'min_price', 'max_price']

missing_cols = [col for col in required_cols if col not in df_slot.columns]
if missing_cols:
    raise ValueError(f"Missing required columns: {missing_cols}")
else:
    print("All required columns are present in df_slot.")

if df_slot.empty:
    raise ValueError("df_slot is empty. This might be due to filters in the previous notebook (e.g., 'Search At >= 2024-08-01'). Please check data processing steps.")

# Handle potential NaNs that could affect clustering/modeling
# For clustering, we'll select specific columns and handle NaNs there.
# For modeling, NaNs in features like lead_time, avg_price should be imputed.
df_slot['lead_time'] = df_slot['lead_time'].fillna(df_slot['lead_time'].median())
df_slot['avg_price'] = df_slot['avg_price'].fillna(df_slot['avg_price'].median())
# pct_avail might also need imputation if used directly
df_slot['pct_avail'] = df_slot['pct_avail'].fillna(df_slot['pct_avail'].mean())


# Ensure booking_rate is not NaN or Inf, cap at 1 for safety if n_bookings > n_searches (unlikely from agg)
df_slot['booking_rate'] = (df_slot['n_bookings'] / df_slot['n_searches']).fillna(0)
df_slot.loc[df_slot['n_searches'] == 0, 'booking_rate'] = 0 # Ensure 0 if no searches
df_slot['booking_rate'] = np.clip(df_slot['booking_rate'], 0, 1) # Ensure it's a probability

print("\nData after initial checks and NaN handling for key modeling features:")
df_slot[required_cols].info()
df_slot.head()

# B. Slot-Level Clustering

In [ ]:
# 1. Feature selection for clustering
# At minimum: booking_rate; optionally add lead_time or avg_price.
# We'll use booking_rate, lead_time, and avg_price.
# Note: avg_price can vary significantly, so scaling is important.
cluster_features = ['booking_rate', 'lead_time', 'avg_price']
df_cluster_data = df_slot[cluster_features + ['Venue Name']].copy()

# Impute NaNs for clustering features if any remain (should be handled above, but as a safeguard)
for col in cluster_features:
    if df_cluster_data[col].isnull().any():
        df_cluster_data[col] = df_cluster_data[col].fillna(df_cluster_data[col].median())

print(f"\nFeatures for clustering: {cluster_features}")
df_cluster_data.head()

# 2. Elbow + silhouette (Example for one venue, then per-venue auto-K)

In [ ]:
# We'll do this within the per-venue loop.

def plot_elbow_silhouette(X_scaled, max_k=10, title_prefix=""):
    inertia = []
    silhouette_scores = []
    
    k_range_inertia = range(1, max_k + 1)
    k_range_silhouette = range(2, max_k + 1)

    for k in k_range_inertia:
        kmeans = KMeans(n_clusters=k, random_state=42, n_init='auto')
        kmeans.fit(X_scaled)
        inertia.append(kmeans.inertia_)

    for k in k_range_silhouette:
        kmeans = KMeans(n_clusters=k, random_state=42, n_init='auto')
        labels = kmeans.fit_predict(X_scaled)
        if len(np.unique(labels)) > 1: # Silhouette score is only defined if there is more than 1 cluster
             silhouette_scores.append(silhouette_score(X_scaled, labels))
        else: # if only one cluster predicted (e.g. k=1 or all points are same)
            silhouette_scores.append(-1) # Invalid score placeholder

    fig, ax1 = plt.subplots(figsize=(10, 5))

    color = 'tab:red'
    ax1.set_xlabel('Number of Clusters (k)')
    ax1.set_ylabel('Inertia', color=color)
    ax1.plot(k_range_inertia, inertia, marker='o', color=color)
    ax1.tick_params(axis='y', labelcolor=color)
    ax1.set_title(f'{title_prefix}Elbow Method & Silhouette Score')

    ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
    color = 'tab:blue'
    ax2.set_ylabel('Silhouette Score', color=color)
    # Align silhouette plot: silhouette_scores has k-2 elements, k_range_silhouette has k-1.
    ax2.plot(k_range_silhouette, silhouette_scores, marker='x', linestyle='--', color=color)
    ax2.tick_params(axis='y', labelcolor=color)

    fig.tight_layout()  # otherwise the right y-label is slightly clipped
    plt.show()
    
    # Determine best k based on silhouette (simple max)
    if silhouette_scores:
        best_k_silhouette = k_range_silhouette[np.argmax(silhouette_scores)]
        print(f"{title_prefix}Suggested k based on max silhouette score: {best_k_silhouette} (score: {max(silhouette_scores):.2f})")
        return best_k_silhouette
    return 3 # Default k if silhouette fails

print("Elbow/Silhouette plotting function defined.")

# 3. Per-venue auto-K and 4. Label semantics

In [ ]:
df_slot_clustered = df_slot.copy()
df_slot_clustered['km_cluster'] = -1 # Initialize cluster column
df_slot_clustered['km_cluster_label'] = 'unknown'

all_venue_clusters_info = [] # To store (venue, date, hour, slot_label, km_cluster)

for venue in df_cluster_data['Venue Name'].unique():
    print(f"\n--- Clustering for Venue: {venue} ---")
    venue_data = df_cluster_data[df_cluster_data['Venue Name'] == venue][cluster_features]
    
    if venue_data.shape[0] < 10: # Not enough data points for meaningful clustering
        print(f"Skipping {venue} due to insufficient data points ({venue_data.shape[0]})")
        # Assign a default cluster or handle as needed
        df_slot_clustered.loc[df_slot_clustered['Venue Name'] == venue, 'km_cluster'] = 0
        df_slot_clustered.loc[df_slot_clustered['Venue Name'] == venue, 'km_cluster_label'] = 'default_low_data'
        
        # Add to all_venue_clusters_info
        venue_slot_indices = df_slot_clustered[df_slot_clustered['Venue Name'] == venue].index
        temp_df = df_slot_clustered.loc[venue_slot_indices, ['Venue Name', 'search_date_for', 'search_hour_for', 'km_cluster_label', 'km_cluster']]
        all_venue_clusters_info.append(temp_df)
        continue

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(venue_data)

    # Determine best k for this venue
    # Set max_k dynamically, e.g., min(10, num_samples-1), but ensure it's at least 2 for silhouette
    max_k_venue = min(10, X_scaled.shape[0]-1 if X_scaled.shape[0] > 1 else 1)
    if max_k_venue < 2 : # Silhouette score requires at least 2 clusters
        print(f"Not enough samples in {venue} for varied k, assigning to single cluster.")
        best_k = 1
        labels = np.zeros(X_scaled.shape[0], dtype=int) # All in one cluster
    else:
        best_k = plot_elbow_silhouette(X_scaled, max_k=max_k_venue, title_prefix=f"{venue}: ")
        if best_k < 2 : best_k = 2 # Ensure at least 2 clusters if silhouette suggested 1 from bad scores
        if best_k > X_scaled.shape[0]-1 : best_k = X_scaled.shape[0]-1 # cap k

        kmeans = KMeans(n_clusters=best_k, random_state=42, n_init='auto')
        labels = kmeans.fit_predict(X_scaled)

    # Store numeric cluster labels for this venue
    venue_slot_indices = df_slot_clustered[df_slot_clustered['Venue Name'] == venue].index
    df_slot_clustered.loc[venue_slot_indices, 'km_cluster'] = labels
    
    # 4. Label semantics: Map numeric clusters to ordered labels based on booking_rate
    temp_venue_df = df_slot_clustered.loc[venue_slot_indices].copy()
    temp_venue_df['numeric_cluster_temp'] = labels # Use the fresh labels
    
    if len(np.unique(labels)) > 1:
        cluster_means = temp_venue_df.groupby('numeric_cluster_temp')['booking_rate'].mean().sort_values()
        
        # Define peak labels dynamically based on number of clusters
        if len(cluster_means) == 1:
            peak_labels = ['standard_peak']
        elif len(cluster_means) == 2:
            peak_labels = ['off_peak', 'peak']
        elif len(cluster_means) == 3:
            peak_labels = ['off_peak', 'mid_peak', 'super_peak']
        else: # For > 3 clusters, use a generic naming scheme
            peak_labels = [f'peak_level_{i}' for i in range(len(cluster_means))]
            
        # Create mapping from sorted numeric cluster to semantic label
        label_mapping = {cluster_idx: peak_labels[i] for i, cluster_idx in enumerate(cluster_means.index)}
        
        df_slot_clustered.loc[venue_slot_indices, 'km_cluster_label'] = temp_venue_df['numeric_cluster_temp'].map(label_mapping)
    else: # Single cluster case
         df_slot_clustered.loc[venue_slot_indices, 'km_cluster_label'] = 'standard_peak'


    # Add to all_venue_clusters_info for CSV export
    temp_df_export = df_slot_clustered.loc[venue_slot_indices, ['Venue Name', 'search_date_for', 'search_hour_for', 'km_cluster_label', 'km_cluster']]
    all_venue_clusters_info.append(temp_df_export)

print("\nPer-venue clustering and label assignment complete.")
df_slot_clustered[['Venue Name', 'km_cluster', 'km_cluster_label', 'booking_rate']].head()


# 5. Save time_slot_clusters.csv

In [ ]:
if all_venue_clusters_info:
    df_time_slot_clusters = pd.concat(all_venue_clusters_info)
    df_time_slot_clusters.to_csv(CLUSTER_CSV, index=False)
    print(f"\nSaved time slot cluster information to {CLUSTER_CSV}")
else:
    print("\nNo cluster information to save (all_venue_clusters_info is empty).")

# Display some cluster stats
print("\nCluster distribution by venue:")
print(df_slot_clustered.groupby(['Venue Name', 'km_cluster_label'])['booking_rate'].agg(['mean', 'count']))


# C. Demand-Prediction Models

In [ ]:
# Prepare data for modeling
# Target: n_bookings (count)
# Features: avg_price, search_hour_for, km_cluster_label (categorical), lead_time, day_of_week, is_weekend, Venue Name, capacity
# km_cluster_label is preferred over km_cluster (numeric) as it has semantic meaning and avoids issues with numeric interpretation.

df_model = df_slot_clustered.copy()

# Ensure target is not NaN and is integer
df_model['n_bookings'] = df_model['n_bookings'].fillna(0).astype(int)

# Define features and target
TARGET = 'n_bookings'

# Numerical features
numerical_features = ['avg_price', 'lead_time', 'capacity'] # search_hour_for, day_of_week, is_weekend can be treated as categorical or numerical
# Categorical features
categorical_features = ['Venue Name', 'km_cluster_label', 'search_hour_for', 'day_of_week', 'is_weekend']


# Handle missing values in features selected for modeling (should be minimal after earlier steps)
for col in numerical_features:
    df_model[col] = df_model[col].fillna(df_model[col].median())
for col in categorical_features:
    df_model[col] = df_model[col].fillna(df_model[col].mode()[0])
    df_model[col] = df_model[col].astype('category') # Ensure they are category type for CatBoost/LGBM


X = df_model[numerical_features + categorical_features]
y = df_model[TARGET]

# Split data (chronological split is often best for time-series related data, but problem asks for rolling window in Optuna)
# For initial baselines, a simple random split. For Optuna, TimeSeriesSplit.
# Using search_date_for to sort for potential time-series split
df_model_sorted = df_model.sort_values(by=['Venue Name', 'search_date_for', 'search_hour_for'])
X_sorted = df_model_sorted[numerical_features + categorical_features]
y_sorted = df_model_sorted[TARGET]

# For baselines, let's use a standard train-test split for simplicity of comparison here.
# For hyperparameter tuning and final model, TimeSeriesSplit will be used.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nData prepared for modeling. X_train shape: {X_train.shape}, X_test shape: {X_test.shape}")
X_train.head()

In [ ]:
# ... later, after creating X_test ... 
# In[8] or equivalent, after X_train, X_test, y_train, y_test = train_test_split(X, y, ...)
print("Unique Venues in X_test:", X_test['Venue Name'].unique()) # <--- ADD THIS
if 'Birmingham' not in X_test['Venue Name'].unique() and 'Birmingham' in df_model['Venue Name'].unique():
    print("WARNING: Birmingham was in df_model but not in X_test. This might be due to the train-test split.")

# 1. Baseline Models

In [ ]:
# Using sklearn's PoissonRegressor and LogisticRegression (for n_bookings > 0)

# For Logistic Regression, target needs to be binary
y_train_binary = (y_train > 0).astype(int)
y_test_binary = (y_test > 0).astype(int)

# Preprocessing for linear models: Scale numerical, OneHotEncode categorical
preprocessor_linear = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), categorical_features) # Ordinal for simplicity here, OHE might be better
    ],
    remainder='passthrough' # Keep any other columns if they exist
)

# --- Logistic Regression (Baseline for P(booking > 0)) ---
pipeline_logreg = Pipeline(steps=[('preprocessor', preprocessor_linear),
                                  ('classifier', LogisticRegression(random_state=42, solver='liblinear', max_iter=1000))])
pipeline_logreg.fit(X_train, y_train_binary)
y_pred_logreg_proba = pipeline_logreg.predict_proba(X_test)[:, 1]
auc_logreg = roc_auc_score(y_test_binary, y_pred_logreg_proba)
print(f"Baseline Logistic Regression AUC (for P(n_bookings > 0)): {auc_logreg:.4f}")

# --- Poisson Regression (Baseline for n_bookings count) ---
# PoissonRegressor doesn't like negative inputs from StandardScaler if features can be zero
# Using a slightly different preprocessor for Poisson or ensuring non-negativity
# For simplicity, let's fit Poisson on data that's not aggressively scaled below zero or use original scales.
# Or, ensure 'capacity' and 'lead_time' are non-negative, 'avg_price' can be tricky if standardized.
# Let's use OrdinalEncoder for categoricals and keep numerical as is or use MinMaxScaler.
preprocessor_poisson = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(with_mean=False), numerical_features), # Scale but don't center, or use MinMaxScaler
        ('cat', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1, 
                               min_frequency=5), # min_frequency to handle rare categories
                               categorical_features)
    ],
    remainder='passthrough'
)

pipeline_poisson = Pipeline(steps=[('preprocessor', preprocessor_poisson),
                                   ('regressor', PoissonRegressor(alpha=1.0, max_iter=1000))]) # alpha is L2 penalty

try:
    X_train_poisson = preprocessor_poisson.fit_transform(X_train)
    X_test_poisson = preprocessor_poisson.transform(X_test)
    
    # PoissonRegressor requires non-negative X if using certain solvers or no intercept.
    # The default 'lbfgs' solver should be fine.
    # If X_train_poisson has negative values after scaling and it causes issues:
    # X_train_poisson[X_train_poisson < 0] = 0 # Simplistic clamp, better to use MinMaxScaler
    
    pipeline_poisson.fit(X_train, y_train) # Fit on original X_train, pipeline handles preprocessing
    y_pred_poisson = pipeline_poisson.predict(X_test)
    
    rmse_poisson = np.sqrt(mean_squared_error(y_test, y_pred_poisson))
    mae_poisson = mean_absolute_error(y_test, y_pred_poisson)
    # Deviance for Poisson: 2 * (sum(y_true * log(y_true/y_pred)) - sum(y_true - y_pred))
    # Sklearn's PoissonRegressor has a score method which is D^2 (explained deviance), not raw deviance.
    # For simplicity, RMSE and MAE are good indicators for count models.
    print(f"Baseline Poisson Regression RMSE: {rmse_poisson:.4f}, MAE: {mae_poisson:.4f}")

except Exception as e:
    print(f"Error in Poisson Regressor: {e}. This might be due to negative values in preprocessed X for Poisson.")
    rmse_poisson, mae_poisson = np.nan, np.nan


baseline_metrics = {
    'LogisticRegression_AUC': auc_logreg,
    'PoissonRegressor_RMSE': rmse_poisson,
    'PoissonRegressor_MAE': mae_poisson,
}

# 2. Gradient-Boosted Trees: LightGBM & CatBoost (Default Params)

In [ ]:
# Target: n_bookings (count) -> Regression task for GBMs

# --- LightGBM ---
# LGBM can handle categorical features directly if specified
# Convert categorical feature names to list of strings for LGBM
lgbm_categorical_features = list(X_train.select_dtypes(include='category').columns)

lgbm_model_default = lgb.LGBMRegressor(random_state=42)
lgbm_model_default.fit(X_train, y_train, categorical_feature=lgbm_categorical_features)
y_pred_lgbm_default = lgbm_model_default.predict(X_test)
rmse_lgbm_default = np.sqrt(mean_squared_error(y_test, y_pred_lgbm_default))
mae_lgbm_default = mean_absolute_error(y_test, y_pred_lgbm_default)
print(f"LightGBM (Default) RMSE: {rmse_lgbm_default:.4f}, MAE: {mae_lgbm_default:.4f}")


# --- CatBoost ---
# CatBoost handles categorical features natively
cat_features_indices = [X_train.columns.get_loc(col) for col in categorical_features]

cb_model_default = cb.CatBoostRegressor(random_state=42, verbose=0, cat_features=cat_features_indices)
cb_model_default.fit(X_train, y_train)
y_pred_cb_default = cb_model_default.predict(X_test)
rmse_cb_default = np.sqrt(mean_squared_error(y_test, y_pred_cb_default))
mae_cb_default = mean_absolute_error(y_test, y_pred_cb_default)
print(f"CatBoost (Default) RMSE: {rmse_cb_default:.4f}, MAE: {mae_cb_default:.4f}")

default_gbm_metrics = {
    'LGBM_Default_RMSE': rmse_lgbm_default,
    'LGBM_Default_MAE': mae_lgbm_default,
    'CatBoost_Default_RMSE': rmse_cb_default,
    'CatBoost_Default_MAE': mae_cb_default,
}

# 3. Hyperparameter Tuning (Optuna)

In [ ]:
# Using TimeSeriesSplit for cross-validation to respect temporal order if applicable
# X_sorted and y_sorted are used here.

# Make sure categorical features in X_sorted are of 'category' dtype for LGBM/CatBoost
for col in categorical_features:
    X_sorted[col] = X_sorted[col].astype('category')

# Time series split
tscv = TimeSeriesSplit(n_splits=3) # Small number of splits for demonstration

# --- Optuna for LightGBM ---
def objective_lgbm(trial):
    # Convert categorical features to codes for this Optuna run if not handled by LGBM directly with 'category' dtype
    # X_train_lgbm, X_val_lgbm = X_sorted.iloc[train_index], X_sorted.iloc[val_index]
    # y_train_lgbm, y_val_lgbm = y_sorted.iloc[train_index], y_sorted.iloc[val_index]
    # Ensure categoricals are handled
    # for col in categorical_features:
    #     X_train_lgbm[col] = X_train_lgbm[col].cat.codes
    #     X_val_lgbm[col] = X_val_lgbm[col].cat.codes
        
    params = {
        'objective': 'poisson', # Good for count data
        'metric': 'rmse',
        'random_state': 42,
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0), # L1
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0), # L2
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
    }
    
    model = lgb.LGBMRegressor(**params)
    
    # Cross-validation using TimeSeriesSplit
    # Optuna integrates with LGBM CV, but manual loop for clarity with TSCV
    rmses = []
    for train_index, val_index in tscv.split(X_sorted):
        X_train_fold, X_val_fold = X_sorted.iloc[train_index], X_sorted.iloc[val_index]
        y_train_fold, y_val_fold = y_sorted.iloc[train_index], y_sorted.iloc[val_index]
        
        model.fit(X_train_fold, y_train_fold, 
                  eval_set=[(X_val_fold, y_val_fold)],
                  eval_metric='rmse',
                  #callbacks=[lgb.early_stopping(50, verbose=False)], # Corrected: early_stopping is a callback
                  callbacks=[lgb.early_stopping(stopping_rounds=50, verbose=-1)],
                  categorical_feature=categorical_features # Pass as list of names
                 )
        preds = model.predict(X_val_fold)
        rmses.append(np.sqrt(mean_squared_error(y_val_fold, preds)))
    
    return np.mean(rmses)

print("\nStarting Optuna for LightGBM...")
study_lgbm = optuna.create_study(direction='minimize')
study_lgbm.optimize(objective_lgbm, n_trials=20) # Reduced trials for speed; increase for better results (e.g., 50-100)
best_params_lgbm = study_lgbm.best_params
print("Best LightGBM params:", best_params_lgbm)

lgbm_tuned = lgb.LGBMRegressor(objective='poisson', metric='rmse', random_state=42, **best_params_lgbm)
lgbm_tuned.fit(X_train, y_train, categorical_feature=categorical_features) # Final fit on X_train, y_train (non-sorted for consistency with test set)
y_pred_lgbm_tuned = lgbm_tuned.predict(X_test)
rmse_lgbm_tuned = np.sqrt(mean_squared_error(y_test, y_pred_lgbm_tuned))
mae_lgbm_tuned = mean_absolute_error(y_test, y_pred_lgbm_tuned)
print(f"LightGBM (Tuned) RMSE: {rmse_lgbm_tuned:.4f}, MAE: {mae_lgbm_tuned:.4f}")


# --- Optuna for CatBoost ---

In [ ]:
# CatBoost needs categorical feature indices
cat_features_indices_sorted = [X_sorted.columns.get_loc(col) for col in categorical_features]

def objective_cb(trial):
    params = {
        'objective': 'RMSE', # CatBoost uses 'RMSE' for regression, Poisson is also an option: 'Poisson'
        'random_seed': 42,
        'verbose': 0,
        'iterations': trial.suggest_int('iterations', 100, 1000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'depth': trial.suggest_int('depth', 3, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-3, 10.0, log=True),
        'border_count': trial.suggest_int('border_count', 32, 255), # For numerical features
         'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.5, 1.0), # Similar to colsample_bytree
    }
    
    model = cb.CatBoostRegressor(**params)
    rmses = []
    for train_index, val_index in tscv.split(X_sorted):
        X_train_fold, X_val_fold = X_sorted.iloc[train_index], X_sorted.iloc[val_index]
        y_train_fold, y_val_fold = y_sorted.iloc[train_index], y_sorted.iloc[val_index]
        
        model.fit(X_train_fold, y_train_fold, 
                  eval_set=[(X_val_fold, y_val_fold)],
                  cat_features=cat_features_indices_sorted, 
                  early_stopping_rounds=50, verbose=0)
        preds = model.predict(X_val_fold)
        rmses.append(np.sqrt(mean_squared_error(y_val_fold, preds)))
        
    return np.mean(rmses)

print("\nStarting Optuna for CatBoost...")
study_cb = optuna.create_study(direction='minimize')
study_cb.optimize(objective_cb, n_trials=20) # Reduced trials for speed
best_params_cb = study_cb.best_params
print("Best CatBoost params:", best_params_cb)

cb_tuned = cb.CatBoostRegressor(objective='RMSE', random_seed=42, verbose=0, **best_params_cb)
cb_tuned.fit(X_train, y_train, cat_features=cat_features_indices) # Final fit on X_train, y_train
y_pred_cb_tuned = cb_tuned.predict(X_test)
rmse_cb_tuned = np.sqrt(mean_squared_error(y_test, y_pred_cb_tuned))
mae_cb_tuned = mean_absolute_error(y_test, y_pred_cb_tuned)
print(f"CatBoost (Tuned) RMSE: {rmse_cb_tuned:.4f}, MAE: {mae_cb_tuned:.4f}")


tuned_gbm_metrics = {
    'LGBM_Tuned_RMSE': rmse_lgbm_tuned,
    'LGBM_Tuned_MAE': mae_lgbm_tuned,
    'CatBoost_Tuned_RMSE': rmse_cb_tuned,
    'CatBoost_Tuned_MAE': mae_cb_tuned,
}

# 4. Calibration & Explainability

In [ ]:
# --- Calibration (for regressors, this is about reliability of point estimates) ---
# Plot predicted vs. actual (binned)
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_cb_tuned, alpha=0.3, label='CatBoost Tuned Predictions')
plt.scatter(y_test, y_pred_lgbm_tuned, alpha=0.3, label='LightGBM Tuned Predictions', marker='x')
plt.plot([min(y_test.min(),0), y_test.max()], [min(y_test.min(),0), y_test.max()], '--', color='red', label='Perfect Prediction')
plt.xlabel("Actual n_bookings")
plt.ylabel("Predicted n_bookings")
plt.title("Predicted vs. Actual n_bookings (Tuned Models)")
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(MODELS_DIR,"predicted_vs_actual.png"))
plt.show()
print("Predicted vs Actual plot saved.")

# Reliability curve is for classifiers. For regressors, we look at residual plots or binned pred vs actual.

# --- SHAP ---
# Using the (potentially) best model, CatBoost Tuned, for SHAP.
# SHAP works with models that output raw scores.
# Need to ensure X_test has numerical categories if CatBoost was trained with integer codes for categories internally.
# However, CatBoost's SHAP implementation usually handles its Pool object well.

# Create a SHAP explainer for CatBoost
# For CatBoost, it's often better to use its internal SHAP value calculation method if available,
# or pass data in the format it expects (e.g., with Pool).
# The `shap.TreeExplainer` works well with CatBoost.
explainer_cb = shap.TreeExplainer(cb_tuned)
shap_values_cb = explainer_cb.shap_values(X_test) # Use the same X_test format as for prediction

plt.figure() # Create a new figure to avoid interference
shap.summary_plot(shap_values_cb, X_test, show=False)
plt.title("SHAP Summary Plot (CatBoost Tuned)")
plt.savefig(os.path.join(MODELS_DIR, "shap_summary.png"), bbox_inches='tight')
plt.show()
print("SHAP summary plot saved to shap_summary.png")


# --- Feature Importance ---
# For CatBoost
feature_importances_cb = pd.DataFrame({
    'feature': X_train.columns,
    'importance_cb': cb_tuned.get_feature_importance()
}).sort_values(by='importance_cb', ascending=False)

# For LightGBM
feature_importances_lgbm = pd.DataFrame({
    'feature': X_train.columns,
    'importance_lgbm': lgbm_tuned.feature_importances_
}).sort_values(by='importance_lgbm', ascending=False)

# Combine and save
feature_importances_combined = pd.merge(feature_importances_lgbm, feature_importances_cb, on='feature', how='outer')
feature_importances_combined = feature_importances_combined.fillna(0) # if a feature was 0 importance in one model
feature_importances_combined.to_csv(os.path.join(MODELS_DIR, "feature_importances.csv"), index=False)
print("\nFeature importances (Top 10 CatBoost):")
print(feature_importances_combined.sort_values(by='importance_cb', ascending=False).head(10))
print(f"Feature importances saved to {os.path.join(MODELS_DIR, 'feature_importances.csv')}")


# 5. Model Selection & Persistence

In [ ]:
# Compare models (RMSE, MAE)
# Lower is better for RMSE and MAE
print("\n--- Model Performance Summary ---")
print("Baseline Metrics:", baseline_metrics)
print("Default GBM Metrics:", default_gbm_metrics)
print("Tuned GBM Metrics:", tuned_gbm_metrics)

# Choose final model (e.g., CatBoost Tuned if it performs best)
# For this example, let's assume CatBoost Tuned is the best.
best_model = cb_tuned
best_model_name = "catboost_tuned"
chosen_model_path = os.path.join(MODELS_DIR, "best_model.pkl")

# Criteria for selection:
# 1. RMSE/MAE (lower is better)
# 2. Calibration (visual inspection of pred vs. actual)
# 3. Inference latency (not measured here, but CatBoost can be slower than LGBM)
# 4. Robustness, ease of use.

# Example: choose based on Tuned CatBoost RMSE vs Tuned LGBM RMSE
if tuned_gbm_metrics['CatBoost_Tuned_RMSE'] < tuned_gbm_metrics['LGBM_Tuned_RMSE']:
    best_model = cb_tuned
    best_model_name = "catboost_tuned"
    print(f"\nSelected CatBoost (Tuned) as the best model with RMSE: {tuned_gbm_metrics['CatBoost_Tuned_RMSE']:.4f}")
else:
    best_model = lgbm_tuned
    best_model_name = "lightgbm_tuned"
    print(f"\nSelected LightGBM (Tuned) as the best model with RMSE: {tuned_gbm_metrics['LGBM_Tuned_RMSE']:.4f}")


joblib.dump(best_model, chosen_model_path)
print(f"Saved best model ({best_model_name}) to {chosen_model_path}")

# Optionally save the other tuned GBM as backup
if best_model_name == "catboost_tuned":
    backup_model = lgbm_tuned
    backup_model_name = "lightgbm_tuned"
else:
    backup_model = cb_tuned
    backup_model_name = "catboost_tuned"

backup_model_path = os.path.join(MODELS_DIR, f"{backup_model_name}_backup.pkl")
joblib.dump(backup_model, backup_model_path)
print(f"Saved backup model ({backup_model_name}) to {backup_model_path}")


# D. Price-Response Grid

# 1. Grid definition

In [ ]:

# Define price range: use min_price/max_price from slot_level, or a dynamic range.
# The `slot_level` data has `min_price` and `max_price` columns, which are fallback bounds.
# `avg_price` is the actual price shown for that slot observation.
# For the grid, we want to vary the `avg_price` feature.

# Let's define a general price range. Example: from £5 to £50 with £1 step.
# A more dynamic way would be venue-specific, using its historical min/max observed prices.
min_sim_price = df_model['avg_price'].min() if not df_model.empty else 5.0
max_sim_price = df_model['avg_price'].max() if not df_model.empty else 50.0
# Ensure min_sim_price is not NaN and is reasonable
min_sim_price = max(5.0, min_sim_price if pd.notna(min_sim_price) else 5.0)
max_sim_price = min(100.0, max_sim_price if pd.notna(max_sim_price) else 50.0) # Cap at 100 for sanity
if min_sim_price >= max_sim_price: max_sim_price = min_sim_price + 10 # Ensure range

price_step = 1.0
simulation_prices = np.arange(min_sim_price, max_sim_price + price_step, price_step)
print(f"\nPrice range for simulation: £{min_sim_price} to £{max_sim_price} with step £{price_step}")



# 2. Expected revenue calculation

In [ ]:

# For each (venue, date, hour) representative slot and each price:
#   predicted_n_bookings = model.predict(featurized_row_with_price)
#   exp_rev = predicted_n_bookings * price

# We need representative slots. Let's pick unique combinations of features OTHER than price.
# Use `df_model` (which contains `X` and `y` before split) as the source of representative slots.
# Keep only one row for each combination of key slot characteristics.
# Key characteristics: Venue Name, search_date_for, search_hour_for, km_cluster_label, day_of_week, is_weekend, capacity
# We will iterate through these unique slots.

# Take a sample of unique slots to avoid huge computation if many unique slots.
# The `search_date_for` makes slots very unique. Let's try to find representative non-date specific slots.
# Or, average features for typical slot types.
# For simplicity, let's use the test set slots (X_test) and vary their 'avg_price'.
# X_test is already a sample.

price_response_data = []

# Ensure X_test_copy has categorical features as 'category' dtype if model expects it
# Best_model (CatBoost/LGBM) can handle 'category' dtype if trained that way.
# X_test used for prediction has original dtypes, which should be fine if model was trained on similar X_train.

print(f"Generating price response grid using {X_test.shape[0]} sample slots from test set...")
counter = 0
for idx, slot_features_orig in X_test.iterrows():
    counter +=1
    if counter % 100 == 0 : print(f"Processing slot {counter}/{X_test.shape[0]}")
    
    slot_features = slot_features_orig.copy() # Base features for this slot
    
    # Original slot characteristics (for grouping/identification)
    venue = slot_features['Venue Name']
    # date = slot_features['search_date_for'] # If 'search_date_for' was in X_test
    # hour = slot_features['search_hour_for'] # If 'search_hour_for' was in X_test
    # These might not be in X_test if they were handled differently (e.g. part of df_model index)
    # Let's assume 'search_date_for' is not a direct feature but can be fetched via index if needed.
    # For this example, we'll just use index from X_test to identify the slot.
    
    original_date = df_model.loc[idx, 'search_date_for']
    original_hour = df_model.loc[idx, 'search_hour_for']

    for price in simulation_prices:
        slot_features['avg_price'] = price # Set the new price
        
        # Ensure feature order and types match what the model expects (DataFrame input)
        feature_row_df = pd.DataFrame([slot_features])
        
        # Predict n_bookings
        # The model 'best_model' expects categorical features to be of dtype 'category' if trained that way.
        # X_train had them converted. Ensure feature_row_df also has them.
        for cat_col in categorical_features:
            if cat_col in feature_row_df.columns:
                 # Ensure it's category type with known categories from training data
                 # This is tricky. Simpler: ensure model's fit on X_train (which had category dtypes) makes it robust.
                 # CatBoost generally handles this well if cat_features indices are provided.
                 # LGBM needs categorical_feature='auto' or specific list of names.
                 # If X_test (and thus slot_features_orig) has them as category, it should be fine.
                 feature_row_df[cat_col] = feature_row_df[cat_col].astype(X_train[cat_col].dtype)


        predicted_n_bookings = best_model.predict(feature_row_df)[0]
        predicted_n_bookings = max(0, predicted_n_bookings) # Ensure non-negative bookings

        expected_revenue = predicted_n_bookings * price
        
        price_response_data.append({
            'slot_id_test_set_idx': idx, # To identify the base slot from X_test
            'Venue Name': venue,
            'search_date_for': original_date, # Original date of the slot from X_test
            'search_hour_for': original_hour, # Original hour
            'km_cluster_label': slot_features['km_cluster_label'],
            'day_of_week': slot_features['day_of_week'],
            'is_weekend': slot_features['is_weekend'],
            'capacity': slot_features['capacity'],
            'simulated_price': price,
            'predicted_n_bookings': predicted_n_bookings,
            'expected_revenue': expected_revenue
        })

df_price_response = pd.DataFrame(price_response_data)
print(f"\nGenerated price response data with {df_price_response.shape[0]} rows.")


# 3. Save price_response_grid.csv

In [ ]:
df_price_response.to_csv(GRID_PATH, index=False)
print(f"Price response grid saved to {GRID_PATH}")
df_price_response.head()


# 4. Visual check: Plot 2-3 sample slots' revenue curves

In [ ]:
# Pick a few unique slot_ids from the grid
if not df_price_response.empty:
    sample_slot_ids = df_price_response['slot_id_test_set_idx'].unique()
    
    num_plots = min(3, len(sample_slot_ids))
    if num_plots > 0:
        plot_ids = np.random.choice(sample_slot_ids, size=num_plots, replace=False)

        fig, axes = plt.subplots(num_plots, 1, figsize=(10, 5 * num_plots), sharex=True)
        if num_plots == 1: axes = [axes] # Make it iterable

        for i, slot_id in enumerate(plot_ids):
            slot_data = df_price_response[df_price_response['slot_id_test_set_idx'] == slot_id]
            ax = axes[i]
            ax.plot(slot_data['simulated_price'], slot_data['expected_revenue'], marker='o', linestyle='-')
            
            # Find peak revenue
            if not slot_data.empty:
                peak_rev_point = slot_data.loc[slot_data['expected_revenue'].idxmax()]
                ax.scatter(peak_rev_point['simulated_price'], peak_rev_point['expected_revenue'], color='red', s=100, zorder=5, 
                           label=f"Peak Rev: £{peak_rev_point['expected_revenue']:.2f} @ £{peak_rev_point['simulated_price']:.2f}")
                ax.legend()

            ax.set_ylabel("Expected Revenue (£)")
            # Include more info in title by fetching from the first row of slot_data
            slot_info_row = slot_data.iloc[0]
            title = (f"Slot: {slot_id} (Venue: {slot_info_row['Venue Name']}, Date: {slot_info_row['search_date_for'].strftime('%Y-%m-%d')}, Hour: {slot_info_row['search_hour_for']})\n"
                     f"Cluster: {slot_info_row['km_cluster_label']}, Cap: {slot_info_row['capacity']}")
            ax.set_title(title)
            ax.grid(True)

        axes[-1].set_xlabel("Simulated Price (£)")
        plt.tight_layout()
        plt.savefig(os.path.join(MODELS_DIR,"sample_revenue_curves.png"))
        plt.show()
        print("Sample revenue curves plotted and saved.")
    else:
        print("Not enough unique slots in price response grid to plot.")
else:
    print("Price response grid is empty, skipping plot.")

print("\n--- Notebook execution complete ---")